# Introduction to the pairinteraction Library

In addition to the [graphical user interface](https://pairinteraction.github.io/images/screen-win64.png), the pairinteraction software can be used as a library. The library provides functionality which can be applied by the user to write its own code, e.g. for searching optimal experimental parameters or calculating effective Hamiltonians and simulating Rydberg experiments.

The library is fully written in C++ to obtain a high performance. It provides a Python wrapper generated with [SWIG](http://www.swig.org) so that one can work with all the functionality of the library in Python. The Python functions are named the same as the wrapped C++ functions so that it is straight forward to transfer code between the two programming languages. The following introduction, which can be [downloaded](https://github.com/pairinteraction/pairinteraction/blob/master/doc/sphinx/examples_python/introduction.ipynb) as a Jupyter notebook, shows the basic usage of the pairinteraction library in Python 3.

Note that, in the unit system used by the pairinteraction library, energy has the unit $\text{GHz}$, length $\mu\text{m}$, the magnetic field $\text{G}$, and the electric field $\text{V/cm}$.





## Preparations

### Installation
By installing a [binary build of the pairinteraction software](https://github.com/pairinteraction/pairinteraction/releases), the pairinteraction library gets installed, too. For GNU/Linux, all dependencies of the pairinteraction library are installed automatically as well, and the library can be used right away. For Windows or OS X, we recommend the installation of the Python 3 distribution [Anaconda](https://www.anaconda.com/distribution/). Then, the dependencies can be installed by executing ``conda install numpy scipy matplotlib pyzmq`` within the Anaconda terminal prompt.

### Importing the Library
Our code starts with loading the required modules for the calculations. We use the module `pireal` of the pairinteraction library as the calculations shown in this introduction require only real numbers (if one considers electric or magnetic fields with a non-zero $y$-value, complex numbers occur due to the definition of the spherical basis and one must use `picomplex`).

In [1]:
# We call an IPython magic function to make the output of plotting commands displayed inline within this notebook.
%matplotlib inline

# Arrays
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Operating system interfaces
import os, sys

# pairinteraction :-)
# For Windows and OS X, we manually add the path containing the library to the Python package search path.
if sys.platform == "darwin": sys.path.append("/Applications/pairinteraction.app/Contents/Resources")
elif sys.platform == "win32": sys.path.append("C:\Program Files\pairinteraction")
from libpairinteraction import pireal as pi

### Creating a Cache for Matrix Elements

The `MatrixElementCache` class provides methods for evaluating matrix elements. Each Python script using the pairinteraction library typically requires one instance of this class. The instance is then passed to every object of the pairinteraction library which needs to evaluates matrix elements. To speed up calculations, the intermediate results of the calculation of matrix elements are cached into memory. If a directory name is passed to the constructor of the class, the specified directory is used to store a SQLite database which holds the intermediate results, making them available to future program runs.

In [2]:
if not os.path.exists("./cache"):
    os.makedirs("./cache")
cache = pi.MatrixElementCache("./cache")

## Defining States
[//]: # (The Rydberg states are specified in the fine structure basis. TODO: discuss atomic species, isotope does not matter)

### Single Atom States

### Pair States

[//]: # (TODO: discuss the two ways for preparing pair states)

## Application 1: Energy Levels and Effective Principal Quantum Numbers

## Application 2: Wave Functions

## Application 3: Matrix Elements
[//]: # (TODO: discuss the units of the matrix elements)

## Application 4: Dispersion Coefficients

## Application 5: Non-perturbative Calculations

For many Rydberg systems, perturbative calculations are either very complictated or not working at all because splittings between energy levels are smaller than interaction energies. This especially happens for short interatomic distances, non-zero interaction angles, or in the presence of electric fields. To study these systems, we must diagonalize their Hamiltonians.

We can apply the pairinteraction library to construct and diagonalize Hamiltonians of Rydberg systems. For systems consisting of one single Rydberg atom, the class `SystemOne` is provided. For two Rydberg atoms, the class `SystemTwo` is given. Both of these classes are used in a similar way: 
* First, we tell which Rydberg states should be considered, letting the software create a **list of relevant Rydberg states**. The Rydberg states can be obtained by calling `System[...].getStates()`.
* Second, we specify whether symmetries of the systems should be taken into account to speed up calculations. The software generates a **list of basis vectors**, where each basis vector is a linear combination of the previously given Rydberg states. If symmetries are specified, they are applied to reduce the size of the basis. The basis vectors can be obtained as the columns of the matrix returned by `System[...].getBasis()`.
* Third, we set up the interactions. The software builds a **matrix representation of the Hamiltonian** in the previously generated basis. The Hamiltonian can be obtained by calling `System[...].getHamiltonian()`.

After calling `System[...].diagonalize()`, the diagonal entries of the Hamiltonian contain the eigenenergies of the system and the list of basis vectors contains the eigenvectors.

[//]: # (TODO: rename the methods specified in SystemBase to meet the nomenclature used in this Jupyter notebook, check whether the columns or rows of System[...].getBasis are the basis vectors)

### Systems Consisting of One Rydberg Atom, Calculating Stark and Zeeman Maps

### Systems Consisting of Two Rydberg Atoms, Calculating Pair Potentials